In [1]:
import numpy as np
import pandas as pd
import healpy as hp

In [2]:
filepath = '/Users/jimenagonzalez/research/DSPL/Hybrid_SL_Modeling/data/candidates_tocompare/Diehl/'

#Reading table2 - Properties of candidate lensing systems
colspecs = [(0,3), (3,13), (14,37), (38,40), (41,46), (47,51), (52,56), (57,81), (82,89)]
table2 = pd.read_fwf( filepath + "table2.dat", colspecs=colspecs,
    names=["Prefix", "DES", "Algorithm", "Rank", "Rad", "e_Rad", "Mult", "OName", "r_OName"])
table2.head()


,Prefix,DES,Algorithm,Rank,Rad,e_Rad,Mult,OName,r_OName
0,DES,J0004-0103,"BNA, 1K, RedM",6.0,2.80,0.29,NaN,NaN,NaN
1,DES,J0006-4208,RNA,6.0,8.42,0.27,NaN,NaN,NaN
2,DES,J0006-4429,1K,5.0,2.30,0.27,NaN,NaN,NaN
3,DES,J0007-4434,"RNA, 1K",7.0,3.50,0.27,NaN,NaN,NaN
4,DES,J0008-5503,1K,4.0,3.63,0.27,NaN,NaN,NaN


In [3]:
#Reading table3 - *Names, positions, photometry, and photometric redshifts of objects for each candidate system

colspecs = [(0, 3),(3, 13),(14, 15),(15, 16),(16, 17),(18, 28),(29, 39),(40, 45),(46, 50),(51, 56),(57, 61), 
        (62, 67), (68, 72), (73, 78), (79, 83), (84, 89), (90, 94), (95, 99), (100, 104) ]
names = ["Prefix", "DES", "Paren1", "m_DES", "Paren2", "RAdeg", "DEdeg", "gmag", "e_gmag", "rmag", "e_rmag",
    "imag", "e_imag", "zmag", "e_zmag", "Ymag", "e_Ymag", "zphot", "e_zphot"]
table3 = pd.read_fwf(filepath + "table3.dat", colspecs=colspecs, names=names)
table3 = table3.drop(columns=["Paren1", "Paren2"])

table3.head()


,Prefix,DES,m_DES,RAdeg,DEdeg,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Ymag,e_Ymag,zphot,e_zphot
0,DES,J0004-0103,A,1.215538,-1.055084,21.46,0.02,20.65,0.02,20.17,0.02,19.89,0.03,19.66,0.05,0.57,0.11
1,DES,J0004-0103,1,1.215325,-1.055798,20.32,0.01,20.18,0.01,20.12,0.02,20.08,0.03,19.89,0.06,0.34,0.06
2,DES,J0004-0103,2,1.214778,-1.055363,21.54,0.01,21.25,0.01,21.08,0.02,21.00,0.04,21.04,0.08,0.45,0.06
3,DES,J0006-4208,A,1.513033,-42.136973,22.93,0.27,20.87,0.05,19.69,0.03,19.11,0.04,18.66,0.07,0.82,0.03
4,DES,J0006-4208,1,1.512545,-42.139283,22.63,0.10,22.30,0.09,21.98,0.11,22.03,0.30,21.45,0.42,0.32,0.10


In [4]:
print(len(table2), len(table3))
print(len(table3[table3['m_DES'] == 'A']))

table3_lenses = table3[(table3['m_DES'] == 'A') | (table3['m_DES'] == 'B')] #Getting only the deflector, not the source images


376 1091
374


In [5]:
#Columns I need for the new dataframe: filename, Rank, AverRad, e_AverRad, RAdeg, DEdeg

new_table = table2.merge(table3_lenses[['RAdeg', 'DEdeg', 'DES']], on = 'DES')
new_table['Name'] = new_table['Prefix'] + ' ' + new_table['DES']
new_table['Rank_norm'] = 3 * (new_table['Rank'] - new_table['Rank'].min()) / (new_table['Rank'].max() - new_table['Rank'].min())
new_table = new_table.rename({'Rad': 'AverRad', 'e_Rad': 'e_AverRad', 'RAdeg': 'RA', 'DEdeg': 'DEC', 'Rank_norm': 'Exp_score'}, axis=1)
new_table = new_table[['Name', 'RA', 'DEC', 'Exp_score', 'AverRad', 'e_AverRad']]

#Adding healpix
ra_rad, dec_rad = np.deg2rad(new_table['RA'].values), np.deg2rad(new_table['DEC'].values)
theta, phi = 0.5 * np.pi - dec_rad, ra_rad 
new_table['HPIX_4096'] = hp.ang2pix(4096, theta, phi, nest=True)

#Removing potential issues
new_table = new_table.fillna(0)
new_table = new_table.astype({'Name': str, 'RA': float, 'DEC': float, 'Exp_score':  float, 'AverRad':  float, 
                             'e_AverRad': float, 'HPIX_4096': int})

new_table.to_csv('Initial_Diehl.csv', index=False)

In [6]:
new_table.dtypes

Name          object
RA           float64
DEC          float64
Exp_score    float64
AverRad      float64
e_AverRad    float64
HPIX_4096      int64
dtype: object